In [18]:
import ollama
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import os
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [24]:
os.getcwd()

'/home/onyxia/work/Ollama'

In [101]:
df=pd.read_csv('/home/onyxia/work/Ollama/data/1.csv')
df.head()

,Contenu,Lien,Source,Unnamed: 0,annotation_id,annotator,created_at,id,lead_time,sentiment,updated_at
0,Accueil Voyage\nSalon Nautique de Paris 2019: ...,http://www.lefigaro.fr/voyages/salon-nautique-...,LF,0,35,1,2024-07-19T20:42:45.261508Z,1,14.644,autre,2024-07-19T20:42:45.261555Z
1,"Après Commercy et Saint-Nazaire, Montceau-les-...",https://www.humanite.fr/social-et-economie/gil...,MANKIND,1,36,1,2024-07-19T20:42:54.225717Z,2,6.048,autre,2024-07-19T20:42:54.225754Z
2,Les conducteurs de locomotives revendiquent un...,https://www.humanite.fr/monde/allemagne/en-all...,MANKIND,2,37,1,2024-07-19T20:43:04.859624Z,3,7.237,grève,2024-07-19T20:43:04.859681Z
3,Accueil Flash Eco\nLe Covid-19 et ses restrict...,http://www.lefigaro.fr/flash-eco/le-covid-19-e...,LF,7,38,1,2024-07-19T20:43:11.420879Z,4,5.103,autre,2024-07-19T20:43:11.420921Z
4,Saintes : grève des psychologues au centre hos...,https://www.sudouest.fr/charente-maritime/la-r...,SOUTHWEST,8,39,1,2024-07-19T20:43:15.843355Z,5,1.838,grève,2024-07-19T20:43:15.843395Z


In [102]:
df = df.dropna(subset=['sentiment'])

In [103]:
df['sentiment'] = df['sentiment'].replace({"grève": 1, "autre": 0})

/tmp/ipykernel_1208/1150596283.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({"grève": 1, "autre": 0})


In [104]:
df=df[['Contenu','sentiment']]

In [122]:
df['Contenu'] = df['Contenu'].str.replace("\n", " ")

In [123]:
df_train, df_test = train_test_split(df, test_size=0.9, random_state=0)

In [57]:
response = ollama.chat(model='llama3.1', messages=[
  {
    'role': 'user',
    'content': 'Is the sky be blue?, I want that you say yes or no',
  },
])
print(response['message']['content'])

Yes.


In [ ]:
ll=ollama.generate(model='llama3.1', prompt='Why is the sky blue?')

In [ ]:
ll['response']

In [120]:
def ReturnResponse(request):
    response_1=ollama.generate(model='llama3.1', prompt="dis moi si " + request + "concene la une greve  et reponds juste par oui ou non; l article doit etre considéré comme greve s il s'agit d'arret de travail, de preavis , d intention de greve ou tout autre chose qui se rapporte à un arrete de travail")
    response_2=ollama.generate(model='llama3.1', prompt="dis moi si " + request + "concene la France ou s il a lieu en France et repond juste par oui ou non Jexige obligatoirement une reponse, si tu douute reponds par oui"  )

    return response_1['response'],response_2['response']

In [15]:
ReturnResponse("""Rentrée difficile après le pont de la Pentecôte pour les usagers des services SNCF en Ile-de-France Le trafic dans Paris et en banlieue sera en effet très fortement perturbé  ce mardi, en raison d' un appel à la grève des cheminots  à la veille d'une journée de négociation sur les primes au titre de leur mobilisation lors des Jeux Olympiques et Paralympiques JOP.La SNCF prévoit notamment un train sur cinq et uniquement aux heures de pointe sur les lignes D et R du réseau Transilien. La ligne E sera également particulièrement affectée avec deux rames sur cinq, sachant que le trafic sera interrompu entre 10 h 00 et 17 h 00, de nombreuses gares n étant pas desservies
""")

('Oui.', 'Oui.')

In [9]:
import re

In [11]:
pattern = re.compile(r'\*\*(\d+)\.\s(.*?)\?\*\*\n\n(.*?)\n\n', re.DOTALL)
matches = pattern.findall(texte)

resultats = {}

for match in matches:
    numero_question = match[0]
    question = match[1]
    reponse = match[2]
    resultats[numero_question] = {
        'question': question,
        'reponse': reponse
    }

for numero, contenu in resultats.items():
    print(f"**{numero}. {contenu['question']}**")
    print(contenu['reponse'])
    print()

**1. S'agit-il d'une grève **
Oui, il s'agit bien d'un appel à la grève des cheminots.

**2. Est-ce que la grève concerne toute la France **
Non, la grève ne semble pas concerner l'ensemble de la France mais plutôt les usagers des services SNCF en Île-de-France (Paris et sa banlieue).

**3. Quelle est l'entreprise concernée **
L'entreprise concernée est la SNCF (Société Nationale des Chemins de Fer Français), qui est l'opérateur ferroviaire français.

**4. Pourquoi a-t-il lieu **
La grève est motivée par un appel à la négociation sur les primes liées à la mobilisation des cheminots lors des Jeux Olympiques et Paralympiques (JOP) organisés en France."



In [124]:
df_train[['Greve', 'France']] = df_train['Contenu'].apply(lambda x: pd.Series(ReturnResponse(x)))

In [125]:
df_train.head()

,Contenu,sentiment,Greve,France
495,"À Saint-Michel-en-Grève, des excréments humain...",0,Non.,Non
38,Réforme de la Justice : journée d'action des a...,0,Oui.,Oui.
396,Négociations salariales : les banques jouent l...,0,Oui,Oui.
58,"Salaires, acquis sociaux, hôpital public : l'u...",0,Oui.,Non.
427,Le festival du 9-3 convoque les grands sorcier...,0,Oui.,Oui


In [ ]:
df_pred=df_train['Greve'].replace({"Oui.": 1, "Non.": 0,'Oui':1,"Non":0})
df_pred.head()

In [143]:
df_train['Greve']=df_train['Greve'].replace({"Oui.": 1, "Non.": 0,'Oui':1,"Non":0})
df_train['France']=df_train['Greve'].replace({"Oui.": 1, "Non.": 0,'Oui':1,"Non":0})

In [ ]:
y_test=df_train["sentiment"]

In [152]:
df1=df_train[['Greve']]
df1.rename(columns={'Greve': 'valeur'}, inplace=True)
df1.head()

/tmp/ipykernel_1208/4182851334.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'Greve': 'valeur'}, inplace=True)


,valeur
495,0
38,1
396,1
58,1
427,1


In [154]:
df2=df_train[['sentiment']]
df2.rename(columns={'sentiment': 'valeur'}, inplace=True)
df2.head()

/tmp/ipykernel_1208/3244160565.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={'sentiment': 'valeur'}, inplace=True)


,valeur
495,0
38,0
396,0
58,0
427,0


In [ ]:
print(confusion_matrix(y_test, df_pred))
print(classification_report(y_test, df_pred))

In [129]:
df_train['Contenu'][396]

"Négociations salariales : les banques jouent la rigueur •Les augmentations générales ne sont pas à l'ordre du jour dans les banques commerciales françaises.•Les établissements se limitent à des mesures annexes. Ajouter à mes articles Commenter Partager Emploi & Salaires Novembre 2015 Par Sharon Wajsbrot , Véronique Chocron Publié le 13 nov. 2015 à 01:01 Dans un climat de très faible inflation, les banques françaises persévèrent en 2016 dans leur politique de rigueur salariale. En effet, les établissements qui achèvent ou ont déjà bouclé leurs négociations salariales annuelles (NAO), comme Société Générale et BNP Paribas, affichent des mesures moins généreuses que celles accordées en 2014. Fait nouveau cette année, Société Générale refuse, selon des sources syndicales, toute mesure générale, qu'il s'agisse de primes ou d'augmentations de salaire pérennes. Pour mémoire, en 2015, les salariés de la banque s'étaient vu octroyer une prime de 200 à 400 euros sous forme de supplément d'intér

In [137]:
y_test

495    0
38     0
396    0
58     0
427    0
487    1
133    1
295    0
28     0
188    1
377    0
467    1
168    1
156    0
249    1
207    1
31     0
32     1
132    0
190    0
270    1
457    1
293    0
430    1
405    1
507    1
152    0
290    0
518    0
182    1
527    0
104    1
345    1
456    0
439    0
342    0
202    1
248    1
472    0
120    1
77     0
179    1
92     0
494    1
319    1
403    1
480    0
75     0
282    0
9      0
366    1
197    0
Name: sentiment, dtype: int64

In [159]:
df1.shape

(52, 1)

In [162]:
def calculer_taux_bonnes_reponses(df1, df2):
    df_merge = df1.join(df2, how='inner', lsuffix='_df1', rsuffix='_df2')
    
    df_merge['bonnes_reponses'] = df_merge['valeur_df1'] == df_merge['valeur_df2']
    
    nombre_bonnes_reponses = df_merge['bonnes_reponses'].sum()
    
    taux_bonnes_reponses = nombre_bonnes_reponses / len(df_merge) * 100
    
    return nombre_bonnes_reponses, taux_bonnes_reponses,df_merge

nombre_bonnes_reponses, taux,df = calculer_taux_bonnes_reponses(df1, df2)

print("Nombre de bonnes réponses :")
print(nombre_bonnes_reponses)

print("\nTaux de bonnes réponses (%) :")
print(taux)


Nombre de bonnes réponses :
23

Taux de bonnes réponses (%) :
44.230769230769226
